In [1]:
import degirum as dg
from dg_eval_tools import *
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm
from model import Model

In [2]:
model_path = "./models/yolov8n_silu_float_6outs.onnx"

ground_truth_annotations_path = "http://100.102.102.108:8084/coco/val/labels.json"
image_folder_path = "http://100.102.102.108:8084/coco/val/data/"

In [3]:
model = Model(model_path, labels='./labels_coco.json')
model.conf_threshold = 0.001
model.iou_threshold = 0.6
class_map = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34,
          35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
          64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90]     

In [4]:
with annotations_accessor(ground_truth_annotations_path) as annotations_file:
    anno = COCO(annotations_file)

files_dict = anno.dataset["images"]
path_list = [os.path.join(image_folder_path, f["file_name"]) for f in files_dict]
img_ids = []

jdict=[]
for image_number, path in tqdm(enumerate(path_list), total=len(path_list)):
    image_id = files_dict[image_number]["id"]
    img_ids.append(image_id)
    predictions = model(path)
    save_results_coco_json(predictions.results,jdict,image_id,class_map)


loading annotations into memory...
Done (t=0.44s)
creating index...
index created!


100%|██████████| 5000/5000 [12:07<00:00,  6.88it/s]


In [5]:
pred_json = os.path.split(model_path)[-1].split('.')[0] + '_predictions.json'
with open(pred_json, 'w+') as f:
    json.dump(jdict, f)

pred = anno.loadRes(pred_json)
eval_obj = COCOeval(anno, pred, "bbox")
eval_obj.params.imgIds = sorted(img_ids) # image IDs to evaluate
eval_obj.evaluate()
eval_obj.accumulate()
eval_obj.summarize()

Loading and preparing results...
DONE (t=3.38s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=33.57s).
Accumulating evaluation results...
DONE (t=5.82s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.349
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.503
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.371
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.153
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.385
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.516
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.286
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.453
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.488
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1